In [ ]:
#pragma cling load("libcurl")
#include "./lib/everest.hpp"
#include <fstream>
{   
    {
        ifstream tokenfile("token");
        if(tokenfile){
            std::cout << "--- аутентификация уже выполнена ---";
            return;
        }
    }
   
    string login,password;
    std::cout << "--- введите ваш login в системе Everest    ---";
    cin >> login;
    std::cout << "--- введите ваш password в системе Everest ---";
    cin >> password;
    
    templet::everest_engine teng(login.c_str(),password.c_str());
    teng.save_access_token();
   
    if (teng) { 
        string token; teng.get_access_token(token); 
        ofstream tokenfile("token");tokenfile << token;
        ofstream loginfile("login");loginfile << login; 
        std::cout << "--- успешная аутентификация ---";}
    else {
        std::cout << "--- сбой аутентификации, попробуйте повторить  ---";
    }
    
    std::cout << endl << "--- для безопасности удалите вывод ячейки ---";
}

In [ ]:
#pragma cling load("libcurl")
#include "./lib/everest.hpp"
#include <fstream>
#include <cstdio>
{
    ifstream tokenfile("token");
    if(!tokenfile){
        std::cout << "--- отсутствует идентификатор открытой сессии ---";
        return;
    }
    string token; tokenfile >> token;
    templet::everest_engine teng(token.c_str());
    teng.save_access_token(false);
    tokenfile.close();
    remove("token");
    std::cout << "--- сессия завершена ---"; 
}

In [ ]:
#pragma cling load("libcurl")
#include "./lib/everest.hpp"
#include <fstream>
{
    ifstream tokenfile("token");
    if(!tokenfile){
        std::cout << "--- отсутствует идентификатор открытой сессии ---" << endl;
        std::cout << "--- откройте новую сессию в системе Everest   ---";
        return;
    }
    string token; tokenfile >> token;
    templet::everest_engine teng(token.c_str());
    
    if(!teng){
        std::cout << "--- возник сбой при использовании текущего идентификатора сессии ---" << endl;
        std::cout << "--- попробуйте снова или закройте сессию затем откройте новую    ---" << endl;
        return;
    }
     
    templet::everest_task save_task(teng,"60bd053d3300003100509cb4");
    
    string file_uri;
    bool res = teng.upload(string("maplecode.txt"),file_uri);
    
    if(!res){
        std::cout << "--- возник сбой при загрузке файла в систему Everest ---" << endl;
        return;
    }
    
    string user;
    ifstream userfile("login"); userfile >> user; 
    
    json input;
    input["name"] = string("user-")+user+string("-save");
    input["inputs"]["user"] = user;
    input["inputs"]["code"] = file_uri;
    
    save_task.submit(input);
    teng.run();
    
    templet::everest_error cntxt;
	if (teng.error(&cntxt)) {
		std::cout << "--- возник сбой при выполнении задачи в системе Everest ---" << std::endl;
		templet::everest_engine::print_error(&cntxt);
		return;
	}
    
    std::cout << "--- maple-программа загружена на компьютер преподавателя ---";
}

In [ ]:
#pragma cling load("libcurl")
#include "./lib/everest.hpp"
#include <fstream>
{
    ifstream tokenfile("token");
    if(!tokenfile){
        std::cout << "--- отсутствует идентификатор открытой сессии ---" << endl;
        std::cout << "--- откройте новую сессию в системе Everest   ---";
        return;
    }
    string token; tokenfile >> token;
    templet::everest_engine teng(token.c_str());
    
    if(!teng){
        std::cout << "--- возник сбой при использовании текущего идентификатора сессии ---" << endl;
        std::cout << "--- попробуйте снова или закройте сессию затем откройте новую    ---" << endl;
        return;
    }
     
    templet::everest_task load_task(teng,"60bd0cfd330000b5eb509cec");
    
    string user;
    ifstream userfile("login"); userfile >> user; 
    
    json input;
    input["name"] = string("user-")+user+string("-load");
    input["inputs"]["user"] = user;
      
    load_task.submit(input);
    teng.run();
    
    templet::everest_error cntxt;
	if (teng.error(&cntxt)) {
		std::cout << "--- возник сбой при выполнении задачи в системе Everest ---" << std::endl;
		templet::everest_engine::print_error(&cntxt);
		return;
	}
    
    json& output = load_task.result();
    bool res = teng.download(string("maplecode.txt"), output["code"]);
    
    if(!res){
        std::cout << "--- возник сбой при загрузке файла в систему JupyterLab ---" << endl;
        return;
    }
    
    std::cout << "--- maple-программа выгружена с компьютера преподавателя в JupyterLab ---" << endl;
    std::cout << "--- обновите файл maplecode.txt через меню File/Reload Text from Disk ---";
}

In [ ]:
#pragma cling load("libcurl")
#include "./lib/everest.hpp"
#include <fstream>
{
    ifstream tokenfile("token");
    if(!tokenfile){
        std::cout << "--- отсутствует идентификатор открытой сессии ---" << endl;
        std::cout << "--- откройте новую сессию в системе Everest   ---";
        return;
    }
    string token; tokenfile >> token;
    templet::everest_engine teng(token.c_str());
    
    if(!teng){
        std::cout << "--- возник сбой при использовании текущего идентификатора сессии ---" << endl;
        std::cout << "--- попробуйте снова или закройте сессию затем откройте новую    ---" << endl;
        return;
    }
    std::cout << "--- идентификатор(ы) everest-приложений для загрузки maple-программ ---" << endl;
    teng.print_app_description("maple-load");
    std::cout << endl << "--- идентификатор(ы) everest-приложений для запуска maple-программ ---" << endl;
    teng.print_app_description("maple-run");
    std::cout << endl << "--- идентификатор(ы) everest-приложений для выгрузки maple-программ ---" << endl;
    teng.print_app_description("maple-save");
}